<a href="https://colab.research.google.com/github/rjkevin/geocoding/blob/main/Geocoding%20Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geocoding: Introduction
Its a computational process that converts physical address to numerical coordinates (Latitude & Longitude)

# Step 1: Install Geopandas & Geopy

In [10]:
!pip install --upgrade geopandas
!pip install --upgrade geopy


Requirement already up-to-date: geopandas in /usr/local/lib/python3.6/dist-packages (0.8.1)
Requirement already up-to-date: geopy in /usr/local/lib/python3.6/dist-packages (2.0.0)


# Step 2: Import Libraries

In [11]:
import pandas as pd 
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster


# Step 3: Identify coordinates using geocoder

Identify coordinates for single address

In [12]:
#Identify coordinates of Eifel Tower

locator = Nominatim(user_agent='myGeocoder')
location = locator.geocode('Champ de Mars, Paris, France')

print('Latitude = {}, Longitude = {}'.format(location.latitude,location.longitude))

Latitude = 48.85614465, Longitude = 2.297820393322227


Idenitfy coordinates for multiple addresses

In [13]:
df = pd.read_csv('/content/addresses.csv')

df.head()

,Unnamed: 0,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon
0,0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89
1,1,Butik,104,NaN,Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,08/662 50 16
2,2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85
3,3,Butik,110,NaN,Hötorgshallen,111 57,STOCKHOLM,Stockholms län,08/56849241
4,4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44


In [14]:
# Concatenate columns address 1, 3,4,5 together

df['ADDRESS'] = df['Address1'].astype(str) + ',' + \
                df['Address3'] + ',' + \
                df['Address4'] + ',' + \
                df['Address5'] + ',' + 'Sweden'

df.head()

,Unnamed: 0,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon,ADDRESS
0,0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89,"Karlaplan 13,115 20,STOCKHOLM,Stockholms län,S..."
1,1,Butik,104,NaN,Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,08/662 50 16,"Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,..."
2,2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85,"Karlavägen 100 A,115 26,STOCKHOLM,Stockholms l..."
3,3,Butik,110,NaN,Hötorgshallen,111 57,STOCKHOLM,Stockholms län,08/56849241,"Hötorgshallen,111 57,STOCKHOLM,Stockholms län,..."
4,4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44,"Drottninggatan 45,111 21,STOCKHOLM,Stockholms ..."


In [15]:
from geopy.extra.rate_limiter import RateLimiter
#1 — We first delay our Geocoding 1 second between each address. This is convenient when you are Geocoding a large number of physical addresses as the Geocoding service provider can deny access to the service.
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

#2 — Create a df['location'] column by applying geocode we created.
df['location'] = df['ADDRESS'].apply(geocode)

#3 — Third, we can create latitude, longitude, and altitude as a single tuple column.
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

#4 — Finally, We split latitude, longitude, and altitude columns into three separate columns.
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [16]:
df.head()

,Unnamed: 0,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon,ADDRESS,location,point,latitude,longitude,altitude
0,0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89,"Karlaplan 13,115 20,STOCKHOLM,Stockholms län,S...","(Karlaplan, Vedbäraren, Östermalm, Östermalms ...","(59.3383148, 18.0899599, 0.0)",59.338315,18.089960,0.0
1,1,Butik,104,NaN,Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,08/662 50 16,"Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,...","(47, Nybrogatan, Brunfisken, Östermalm, Österm...","(59.3372072, 18.0790982, 0.0)",59.337207,18.079098,0.0
2,2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85,"Karlavägen 100 A,115 26,STOCKHOLM,Stockholms l...","(Karlavägen, Batteriet, Östermalm, Östermalms ...","(59.3358717, 18.0995103, 0.0)",59.335872,18.099510,0.0
3,3,Butik,110,NaN,Hötorgshallen,111 57,STOCKHOLM,Stockholms län,08/56849241,"Hötorgshallen,111 57,STOCKHOLM,Stockholms län,...","(Hötorgshallen, Hötorget, Konserthuset, Norrma...","(59.3343273, 18.0626039, 0.0)",59.334327,18.062604,0.0
4,4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44,"Drottninggatan 45,111 21,STOCKHOLM,Stockholms ...","(Drottninggatan, Klarabergsgatan, Norrmalm, No...","(59.3324377, 18.0626176, 0.0)",59.332438,18.062618,0.0


In [17]:
df

,Unnamed: 0,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon,ADDRESS,location,point,latitude,longitude,altitude
0,0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89,"Karlaplan 13,115 20,STOCKHOLM,Stockholms län,S...","(Karlaplan, Vedbäraren, Östermalm, Östermalms ...","(59.3383148, 18.0899599, 0.0)",59.338315,18.089960,0.0
1,1,Butik,104,NaN,Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,08/662 50 16,"Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,...","(47, Nybrogatan, Brunfisken, Östermalm, Österm...","(59.3372072, 18.0790982, 0.0)",59.337207,18.079098,0.0
2,2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85,"Karlavägen 100 A,115 26,STOCKHOLM,Stockholms l...","(Karlavägen, Batteriet, Östermalm, Östermalms ...","(59.3358717, 18.0995103, 0.0)",59.335872,18.099510,0.0
3,3,Butik,110,NaN,Hötorgshallen,111 57,STOCKHOLM,Stockholms län,08/56849241,"Hötorgshallen,111 57,STOCKHOLM,Stockholms län,...","(Hötorgshallen, Hötorget, Konserthuset, Norrma...","(59.3343273, 18.0626039, 0.0)",59.334327,18.062604,0.0
4,4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44,"Drottninggatan 45,111 21,STOCKHOLM,Stockholms ...","(Drottninggatan, Klarabergsgatan, Norrmalm, No...","(59.3324377, 18.0626176, 0.0)",59.332438,18.062618,0.0
5,5,Butik,114,PK-Huset,Norrlandsgatan 3,111 47,STOCKHOLM,Stockholms län,08/796 98 10,"Norrlandsgatan 3,111 47,STOCKHOLM,Stockholms l...","(Norrlandsgatan, Packarhuset, Norrmalm, Norrma...","(59.3329989, 18.071481, 0.0)",59.332999,18.071481,0.0
6,6,Butik,116,NaN,Drottninggatan 22,111 51,STOCKHOLM,Stockholms län,08/21 22 60,"Drottninggatan 22,111 51,STOCKHOLM,Stockholms ...","(22, Drottninggatan, Elefanten Mindre, Norrmal...","(59.3304823, 18.0649707, 0.0)",59.330482,18.064971,0.0
7,7,Butik,132,Marieberg,Rålambsvägen 7-9,112 59,STOCKHOLM,Stockholms län,08/13 30 95,"Rålambsvägen 7-9,112 59,STOCKHOLM,Stockholms l...",None,None,NaN,NaN,NaN
8,8,Butik,133,NaN,Kungsholmstorg 11 A,112 21,STOCKHOLM,Stockholms län,08/653 39 17,"Kungsholmstorg 11 A,112 21,STOCKHOLM,Stockholm...","(Kungsholmstorg, Kungsholmen, Kungsholmens sta...","(59.32886965, 18.04308918335665, 0.0)",59.328870,18.043089,0.0
9,9,Butik,134,NaN,Lindhagensgatan 118,112 51,STOCKHOLM,Stockholms län,08/656 14 30,"Lindhagensgatan 118,112 51,STOCKHOLM,Stockholm...","(118, Lindhagensgatan, Lyckan, Stadshagen, Kun...","(59.3373898, 18.009267, 0.0)",59.337390,18.009267,0.0


In [18]:
df.latitude.isnull().sum()

2

In [19]:
df = df[pd.notnull(df["latitude"])]

df

,Unnamed: 0,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon,ADDRESS,location,point,latitude,longitude,altitude
0,0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89,"Karlaplan 13,115 20,STOCKHOLM,Stockholms län,S...","(Karlaplan, Vedbäraren, Östermalm, Östermalms ...","(59.3383148, 18.0899599, 0.0)",59.338315,18.089960,0.0
1,1,Butik,104,NaN,Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,08/662 50 16,"Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,...","(47, Nybrogatan, Brunfisken, Östermalm, Österm...","(59.3372072, 18.0790982, 0.0)",59.337207,18.079098,0.0
2,2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85,"Karlavägen 100 A,115 26,STOCKHOLM,Stockholms l...","(Karlavägen, Batteriet, Östermalm, Östermalms ...","(59.3358717, 18.0995103, 0.0)",59.335872,18.099510,0.0
3,3,Butik,110,NaN,Hötorgshallen,111 57,STOCKHOLM,Stockholms län,08/56849241,"Hötorgshallen,111 57,STOCKHOLM,Stockholms län,...","(Hötorgshallen, Hötorget, Konserthuset, Norrma...","(59.3343273, 18.0626039, 0.0)",59.334327,18.062604,0.0
4,4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44,"Drottninggatan 45,111 21,STOCKHOLM,Stockholms ...","(Drottninggatan, Klarabergsgatan, Norrmalm, No...","(59.3324377, 18.0626176, 0.0)",59.332438,18.062618,0.0
5,5,Butik,114,PK-Huset,Norrlandsgatan 3,111 47,STOCKHOLM,Stockholms län,08/796 98 10,"Norrlandsgatan 3,111 47,STOCKHOLM,Stockholms l...","(Norrlandsgatan, Packarhuset, Norrmalm, Norrma...","(59.3329989, 18.071481, 0.0)",59.332999,18.071481,0.0
6,6,Butik,116,NaN,Drottninggatan 22,111 51,STOCKHOLM,Stockholms län,08/21 22 60,"Drottninggatan 22,111 51,STOCKHOLM,Stockholms ...","(22, Drottninggatan, Elefanten Mindre, Norrmal...","(59.3304823, 18.0649707, 0.0)",59.330482,18.064971,0.0
8,8,Butik,133,NaN,Kungsholmstorg 11 A,112 21,STOCKHOLM,Stockholms län,08/653 39 17,"Kungsholmstorg 11 A,112 21,STOCKHOLM,Stockholm...","(Kungsholmstorg, Kungsholmen, Kungsholmens sta...","(59.32886965, 18.04308918335665, 0.0)",59.328870,18.043089,0.0
9,9,Butik,134,NaN,Lindhagensgatan 118,112 51,STOCKHOLM,Stockholms län,08/656 14 30,"Lindhagensgatan 118,112 51,STOCKHOLM,Stockholm...","(118, Lindhagensgatan, Lyckan, Stadshagen, Kun...","(59.3373898, 18.009267, 0.0)",59.337390,18.009267,0.0
10,11,Butik,137,NaN,Fleminggatan 56,112 45,STOCKHOLM,Stockholms län,08/653 10 58,"Fleminggatan 56,112 45,STOCKHOLM,Stockholms lä...","(56, Fleminggatan, Knoppen, Kungsholmen, Kungs...","(59.3349499, 18.0300502, 0.0)",59.334950,18.030050,0.0


# Step 4: Plot coordinates on maps using folium 

In [20]:

map1 = folium.Map(
    location=[59.338315,18.089960],
    tiles='cartodbpositron',
    zoom_start=12,
)

df.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1

In [21]:
map1.save("map1.html")

In [22]:
folium_map = folium.Map(location=[59.338315,18.089960],
                        zoom_start=12,
                        tiles='CartoDB dark_matter')


FastMarkerCluster(data=list(zip(df['latitude'].values, df['longitude'].values))).add_to(folium_map)
folium.LayerControl().add_to(folium_map)
folium_map

In [23]:
folium_map.save("map2.html")